In [1]:
import numpy as np
import pandas as pd

from scipy.stats import spearmanr

from sklearn.model_selection import train_test_split

from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from sklearn import ensemble

In [2]:
# Functions

def metric_train(output_predictions, output_exact):

    return  spearmanr(output_predictions, output_exact).correlation

### Load the data

In [3]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')

X_test = pd.read_csv('X_test.csv')

print('-----------------------------------')
print('X_train shape:\t\t', X_train.shape)
print('y_train shape:\t\t', y_train.shape)
print('-----------------------------------')
print('X_test shape:\t\t', X_test.shape)
print('-----------------------------------')

-----------------------------------
X_train shape:		 (1494, 35)
y_train shape:		 (1494, 2)
-----------------------------------
X_test shape:		 (654, 35)
-----------------------------------


### Look & manipulate the data 

In [4]:
X_train.head()

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,FR_RESIDUAL_LOAD,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET
0,1054,206,FR,0.210099,-0.427458,-0.606523,0.606523,NaN,0.692860,NaN,...,-0.444661,-0.172680,-0.556356,-0.790823,-0.283160,-1.069070,-0.063404,0.339041,0.124552,-0.002445
1,2049,501,FR,-0.022399,-1.003452,-0.022063,0.022063,-0.573520,-1.130838,0.573520,...,-1.183194,-1.240300,-0.770457,1.522331,0.828412,0.437419,1.831241,-0.659091,0.047114,-0.490365
2,1924,687,FR,1.395035,1.978665,1.021305,-1.021305,-0.622021,-1.682587,0.622021,...,1.947273,-0.480700,-0.313338,0.431134,0.487608,0.684884,0.114836,0.535974,0.743338,0.204952
3,297,720,DE,-0.983324,-0.849198,-0.839586,0.839586,-0.270870,0.563230,0.270870,...,-0.976974,-1.114838,-0.507570,-0.499409,-0.236249,0.350938,-0.417514,0.911652,-0.296168,1.073948
4,1101,818,FR,0.143807,-0.617038,-0.924990,0.924990,NaN,0.990324,NaN,...,-0.526267,-0.541465,-0.424550,-1.088158,-1.011560,0.614338,0.729495,0.245109,1.526606,2.614378


In [5]:
y_train.head()

,ID,TARGET
0,1054,0.028313
1,2049,-0.112516
2,1924,-0.180840
3,297,-0.260356
4,1101,-0.071733


In [6]:
print("COLUMNS HEADINGS:\n", X_train.columns.values)

COLUMNS HEADINGS:
 ['ID' 'DAY_ID' 'COUNTRY' 'DE_CONSUMPTION' 'FR_CONSUMPTION'
 'DE_FR_EXCHANGE' 'FR_DE_EXCHANGE' 'DE_NET_EXPORT' 'FR_NET_EXPORT'
 'DE_NET_IMPORT' 'FR_NET_IMPORT' 'DE_GAS' 'FR_GAS' 'DE_COAL' 'FR_COAL'
 'DE_HYDRO' 'FR_HYDRO' 'DE_NUCLEAR' 'FR_NUCLEAR' 'DE_SOLAR' 'FR_SOLAR'
 'DE_WINDPOW' 'FR_WINDPOW' 'DE_LIGNITE' 'DE_RESIDUAL_LOAD'
 'FR_RESIDUAL_LOAD' 'DE_RAIN' 'FR_RAIN' 'DE_WIND' 'FR_WIND' 'DE_TEMP'
 'FR_TEMP' 'GAS_RET' 'COAL_RET' 'CARBON_RET']


#### Remove the repeted values from X_train

We can see that for some days we have the same informations twice, once for the country FR and once for the country DE, but all the values in the other columns are the same.
To see this we first order the DF by day, and then we drop all the rows that have the same DAY_ID.
Furthermore, lets drop the Columns "COUNTRY" and "DAY_ID" since it doesn't give us any additions info. All we will need to match the features with the labels will be the "ID".

In [7]:
X_train.sort_values(by=['DAY_ID'], inplace=True)
X_train.head(10)

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,FR_RESIDUAL_LOAD,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET
742,1057,0,FR,0.048418,-0.465067,-0.079296,0.079296,NaN,0.650577,NaN,...,-0.344706,-0.987968,-0.856204,-0.655528,-1.099226,0.279885,0.583874,-1.071996,-0.074767,0.798003
1211,1179,1,FR,0.966997,1.222131,0.331356,-0.331356,0.405549,0.778627,-0.405549,...,1.214288,-0.370990,-0.497520,-1.184018,-1.465608,1.349212,0.231602,1.480313,0.931562,0.822047
1408,1327,2,FR,-0.068972,-0.667390,-1.102015,1.102015,-1.080403,0.256736,1.080403,...,-0.540642,-0.344600,-0.372156,-0.623041,-0.926064,-0.148950,0.641235,1.802550,1.140920,0.900434
1468,111,2,DE,-0.068972,-0.667390,-1.102015,1.102015,-1.080403,0.256736,1.080403,...,-0.540642,-0.344600,-0.372156,-0.623041,-0.926064,-0.148950,0.641235,1.802550,1.140920,0.900434
329,2016,3,FR,-0.134670,-0.834564,-1.051716,1.051716,-1.881881,-0.612133,1.881881,...,-0.856321,-1.394561,-1.118297,0.244859,0.488650,-1.710888,-0.951057,0.440121,-0.064550,-0.032756
680,800,3,DE,-0.134670,-0.834564,-1.051716,1.051716,-1.881881,-0.612133,1.881881,...,-0.856321,-1.394561,-1.118297,0.244859,0.488650,-1.710888,-0.951057,0.440121,-0.064550,-0.032756
512,2047,5,FR,-0.297850,-0.470371,0.144615,-0.144615,-1.208286,-1.811403,1.208286,...,-0.552878,-1.374757,-0.790071,0.333817,0.021868,-0.062187,1.459745,-0.117977,0.550433,0.781870
991,831,5,DE,-0.297850,-0.470371,0.144615,-0.144615,-1.208286,-1.811403,1.208286,...,-0.552878,-1.374757,-0.790071,0.333817,0.021868,-0.062187,1.459745,-0.117977,0.550433,0.781870
592,1995,7,FR,0.057599,-0.625625,0.002239,-0.002239,-0.676226,-0.745182,0.676226,...,-0.806379,-0.531862,-0.663419,0.098940,1.367421,-0.812164,0.954384,-0.379980,0.518459,-0.034642
1356,779,7,DE,0.057599,-0.625625,0.002239,-0.002239,-0.676226,-0.745182,0.676226,...,-0.806379,-0.531862,-0.663419,0.098940,1.367421,-0.812164,0.954384,-0.379980,0.518459,-0.034642


In [8]:
X_train.drop_duplicates(subset=['DAY_ID'], inplace=True)
X_train.drop(['COUNTRY', "DAY_ID"], axis=1, inplace=True)

# Oder by ID and reset index
X_train.sort_values(by=['ID'], inplace=True)
X_train.reset_index(drop=True, inplace=True)
X_train.head(10)

,ID,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,FR_NET_IMPORT,DE_GAS,...,FR_RESIDUAL_LOAD,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET
0,4,0.284447,0.356063,0.389806,-0.389806,1.506291,-0.035824,-1.506291,0.035824,0.202693,...,0.158859,-0.300883,-0.148592,-0.444411,-0.622197,-0.795390,-1.352885,-1.951199,0.273857,-1.847703
1,5,0.551274,0.256555,0.226737,-0.226737,1.205343,0.067784,-1.205343,-0.067784,0.637224,...,0.335134,-0.347601,0.035837,-0.530196,-0.896302,-0.511760,-0.562027,-0.307876,0.225957,-0.408372
2,10,0.263204,-0.218111,-1.224154,1.224154,-1.093566,0.947850,1.093566,-0.947850,0.058100,...,-0.482298,-0.122068,-0.309827,-0.502343,-0.224969,1.273429,-0.419321,0.175473,-0.139109,0.745399
3,12,0.320268,-0.107240,-1.106101,1.106101,-0.830674,0.783837,0.830674,-0.783837,0.370300,...,-0.014458,0.301802,-0.663711,-0.276413,-0.694926,-1.045810,-0.780288,-0.302094,0.638991,0.192146
4,14,0.181137,-0.296033,-0.216369,0.216369,0.028765,0.536853,-0.028765,-0.536853,-0.027636,...,-0.227884,-0.856498,-0.522646,-0.164743,-0.633300,-0.121444,-0.276122,0.697493,-1.297575,0.782801
5,17,0.480896,-0.007138,-0.603067,0.603067,-0.920045,0.544789,0.920045,-0.544789,0.537622,...,0.082377,-0.643678,-0.194508,-0.800195,-0.744488,-1.674473,-1.653002,-0.940520,1.378494,-0.459267
6,18,0.579304,-0.558880,-1.549859,1.549859,-0.397575,1.457395,0.397575,-1.457395,0.328400,...,-0.850719,0.072652,-0.054172,-0.451614,-0.184350,-1.434130,-0.996202,0.830346,0.631591,0.433844
7,19,0.472095,-0.107581,-1.252716,1.252716,-0.402936,1.292833,0.402936,-1.292833,0.425866,...,-0.431427,1.027283,-0.173300,-0.502079,-0.178360,-0.825313,-1.400584,-1.177699,-0.501738,0.346499
8,21,0.157215,-0.311642,-1.716146,1.716146,-0.617396,1.313862,0.617396,-1.313862,-0.085741,...,-0.785984,-0.511357,-0.565777,-0.273137,0.238254,-1.029980,-0.541625,-0.918264,-0.752650,-0.620486
9,23,0.484932,-0.376975,-1.468259,1.468259,-0.745371,1.191019,0.745371,-1.191019,0.071106,...,-0.619978,-0.664240,-0.309580,-0.422287,-0.433411,-1.291306,-0.224399,0.000000,0.100383,0.195996


#### Adjust y_train to match X_train

We have the same repeted vaules in __y_train__ therefore we're going to remove all those days. To do this we will simply keep the __ID__ that is still prensent in __X_train__.

In [9]:
y_train = y_train[y_train["ID"].isin(X_train["ID"])]

# Order Y_train by DAY_ID and reset index
y_train.sort_values(['ID'], inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_train.head(10)

,ID,TARGET
0,4,0.029822
1,5,-0.073464
2,10,0.324191
3,12,1.051027
4,14,1.475031
5,17,-0.327873
6,18,0.441189
7,19,-0.030627
8,21,0.129770
9,23,-0.082602


#### Clean the dataset

Here we are going to remove all the NaN values and the columns that do not have relavant informartion for the model.

In [10]:
X_train_clean = X_train.fillna(0).drop(['ID'], axis=1)
y_train_clean = y_train['TARGET']

X_test_clean = X_test.fillna(0).drop(['ID', 'DAY_ID', 'COUNTRY'], axis=1)

### Gradient Boosting Method

To create the model we are going to use __Gradient Boosting Method__. This because we do not have many data and this methods will help us also to avoid to overfit our predictions.

In [12]:
# Parameters for Gradient Boosting Regressor
params = {
    "n_estimators": 1000,
    "max_depth": 20,
    "min_samples_split": 5,
    "learning_rate": 0.001,
    "loss": "squared_error",
    }

#### Test the Grad Boosting Method splittind the dataset in train and test

In [13]:
X_train_batch, X_test_batch, y_train_batch, y_test_batch = train_test_split(X_train_clean, y_train_clean,
                                                                            test_size=0.1,
                                                                            random_state=13
                                                                            )

# Gradient Boosting Regressor
reg = ensemble.GradientBoostingRegressor(**params)

# Fit the model
reg.fit(X_train_batch, y_train_batch)

GradientBoostingRegressor(learning_rate=0.001, max_depth=20,
                          min_samples_split=5, n_estimators=1000)

In [13]:
predition_y_test_batch = reg.predict(X_test_batch)

print('Spearman correlation for the train set: {:.1f}%'.format(100 * metric_train(predition_y_test_batch, y_test_batch) ))

Spearman correlation for the train set: 27.3%


#### Train the Gradient Boosting Method using the full dataset & make the predictions

In [14]:
# Gradient Boosting Regressor
reg = ensemble.GradientBoostingRegressor(**params)

# Fit the model
reg.fit(X_train_clean, y_train_clean)

GradientBoostingRegressor(learning_rate=0.001, max_depth=20,
                          min_samples_split=5, n_estimators=1000)

In [15]:
# Make the predictions
predictions_y_test = reg.predict(X_test_clean)

# Define the predictions dataframe
y_test = pd.DataFrame(columns=['ID', 'TARGET'])
y_test['ID'] = X_test['ID']
y_test['TARGET'] = predictions_y_test

y_test.head()

,ID,TARGET
0,1115,0.034659
1,1202,0.029804
2,1194,0.010156
3,1084,0.138112
4,1135,0.198859


In [17]:
# Save the output dataframe
y_test.to_csv('predictions.csv', index=False)

### Use Optuna to find the best parameters

In [21]:
import optuna
# Set optuna verbosity
optuna.logging.set_verbosity(optuna.logging.WARNING)

X_train_batch, X_test_batch, y_train_batch, y_test_batch = train_test_split(X_train_clean, y_train_clean,
                                                                            test_size=0.1,
                                                                            random_state=13
                                                                            )

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 500, 1500)
    max_depth = trial.suggest_int('max_depth', 10, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 5, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.0001, 0.01)

    params = {
        "n_estimators": n_estimators,
        "max_depth": max_depth,
        "min_samples_split": min_samples_split,
        "learning_rate": learning_rate,
        "loss": "squared_error",
        }
    
    reg = ensemble.GradientBoostingRegressor(**params)
    reg.fit(X_train_batch, y_train_batch)
    predition_y_test_batch = reg.predict(X_test_batch)

    return metric_train(predition_y_test_batch, y_test_batch)*100

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=25, show_progress_bar=True)

study.best_params

  0%|          | 0/5 [00:00<?, ?it/s]

{'n_estimators': 971,
 'max_depth': 15,
 'min_samples_split': 4,
 'learning_rate': 0.0076822470599879344}